In [1]:
import pandas as pd
import numpy as np
import re

import nltk
import nltk.data

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

from nltk import pos_tag, word_tokenize
from nltk.corpus.reader import WordListCorpusReader

from corus import load_lenta
from tqdm.notebook import tqdm

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [28]:
from nltk.tokenize import sent_tokenize, word_tokenize

#### Создам корпус русского языка на основе новостного портала лента.ру

In [2]:
path = 'J:/Storage_for_ML/lenta-ru-news.csv.gz'
records = load_lenta(path)
data = [(record.title, record.topic, record.text, record.tags) for record in records]
data = pd.DataFrame(data, columns=['title','topic','text','tags'])
data.drop(columns = ['title', 'topic', 'tags'], axis = 1, inplace=True)
data = data.sample(frac=0.5)
data.sample(3)

,text
422545,"Общий объем средств, потраченных россиянами в ..."
265251,Бразильская авиастроительная компания Embraer ...
18218,Израиль эвакуировал из Сирии активистов «Белых...


In [5]:
data.text.iloc[10]

'Верховный суд отменил оправдательный приговор по делу об убийстве в подмосковной электричке уроженца Армении Артура Сардаряна, сообщает "Эхо Москвы". Адвокаты потерпевших и гособвинитель утверждали, что в суде первой инстанции были нарушены нормы уголовно-процессуального законодательства. Верховный суд признал правоту потерпевших и обвинения и удовлетворил их жалобу. По словам адвоката потерпевших Симона Цатуряна, дело будет направлено на новое рассмотрение в Московский областной суд. 19-летний Артур Сардарян был убит 25 мая 2006 года. Он ехал на электричке домой в Пушкино по Ярославской железной дороге. Когда поезд находился рядом со станцией "Клязьма", в вагон вошли зашли двое молодых людей, один из которых подбежал к юноше и с криками "Слава России!" со спины нанес ему ножом не менее пяти ударов в голову и шею. Вскоре сотрудники правоохранительных органов задержали одного из нападавших. Против него было возбуждено уголовное дело по пункту "л" части 2 статьи 105 Уголовного кодекса (

#### Создам отдельный файл содержащий все новости. это будет мой корпус на русском языке.

In [7]:
data_sent = [el for el in data.text]
data_sent[:3]

['Челябинский танковый институт, где в начале января 2006 года старослужащие искалечили новобранца, будет расформирован. Как сообщает информационное агентство РИА Новости, об этом заявил журналистам заместитель главнокомандующего Сухопутными войсками генерал-полковник Владимир Молтенской. Однако генерал-полковник утверждает, что расформировано учебное заведение будет только "в перспективе". Точной даты он не назвал. Более того, по словам заместителя Главкома, предстоящая ликвидация института никак не связана с издевательствами над новобранцами. Молтенской сказал, что это произойдет в соответствии с общим планом реформирования Российских вооруженных сил. "Челябинский институт не совсем благополучен в плане профессиональной подготовки будущих офицеров-танкистов, и еще в прошлом году было внесено в план оптимизации вузов Сухопутных войск, который начнет осуществляться с 2006 года. Но после известного чрезвычайного и беспрецедентного происшествия процесс расформирование училища, видимо, бу

In [8]:
count = 0
with open('C:/Users/Вадим/nltk_data/corpora/data.txt', "w", encoding='utf-8') as file:
    for el in data_sent:
        print(el, file=file, sep="\n")

In [9]:
with open('C:/Users/Вадим/nltk_data/corpora/data.txt', "r", encoding='utf-8') as file:
    list_sent = [line.rstrip() for line in file if line]

In [72]:
list_sent[:1]

['Челябинский танковый институт, где в начале января 2006 года старослужащие искалечили новобранца, будет расформирован. Как сообщает информационное агентство РИА Новости, об этом заявил журналистам заместитель главнокомандующего Сухопутными войсками генерал-полковник Владимир Молтенской. Однако генерал-полковник утверждает, что расформировано учебное заведение будет только "в перспективе". Точной даты он не назвал. Более того, по словам заместителя Главкома, предстоящая ликвидация института никак не связана с издевательствами над новобранцами. Молтенской сказал, что это произойдет в соответствии с общим планом реформирования Российских вооруженных сил. "Челябинский институт не совсем благополучен в плане профессиональной подготовки будущих офицеров-танкистов, и еще в прошлом году было внесено в план оптимизации вузов Сухопутных войск, который начнет осуществляться с 2006 года. Но после известного чрезвычайного и беспрецедентного происшествия процесс расформирование училища, видимо, бу

In [92]:
reader_corpus = WordListCorpusReader('.', ['C:/Users/Вадим/nltk_data/corpora/data.txt'])

---

In [14]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [15]:
corpus_root = 'C:/Users/Вадим/nltk_data/corpora/'
newcorpus = PlaintextCorpusReader(corpus_root, '.*')

In [18]:
no_of_words_corpus = len(newcorpus.words("data.txt"))

In [19]:
no_of_words_corpus

81228431

---

In [22]:
def get_pos(sentens):
    res = pos_tag(word_tokenize(sentens), lang='rus')
    return res

In [83]:
list_sent_pos = []
for el in list_sent:
    list_sent_pos.append(get_pos(el))

In [82]:
list_sent_pos[:1]

[[('Челябинский', 'A=m'),
  ('танковый', 'A=m'),
  ('институт', 'S'),
  (',', 'NONLEX'),
  ('где', 'ADV-PRO'),
  ('в', 'PR'),
  ('начале', 'S'),
  ('января', 'S'),
  ('2006', 'NUM=ciph'),
  ('года', 'S'),
  ('старослужащие', 'V'),
  ('искалечили', 'V'),
  ('новобранца', 'S'),
  (',', 'NONLEX'),
  ('будет', 'V'),
  ('расформирован', 'V'),
  ('.', 'NONLEX'),
  ('Как', 'ADV-PRO'),
  ('сообщает', 'V'),
  ('информационное', 'A=n'),
  ('агентство', 'S'),
  ('РИА', 'S'),
  ('Новости', 'S'),
  (',', 'NONLEX'),
  ('об', 'PR'),
  ('этом', 'S-PRO'),
  ('заявил', 'V'),
  ('журналистам', 'S'),
  ('заместитель', 'S'),
  ('главнокомандующего', 'V'),
  ('Сухопутными', 'A=pl'),
  ('войсками', 'S'),
  ('генерал-полковник', 'S'),
  ('Владимир', 'S'),
  ('Молтенской', 'S'),
  ('.', 'NONLEX'),
  ('Однако', 'CONJ'),
  ('генерал-полковник', 'S'),
  ('утверждает', 'V'),
  (',', 'NONLEX'),
  ('что', 'CONJ'),
  ('расформировано', 'V'),
  ('учебное', 'A=n'),
  ('заведение', 'S'),
  ('будет', 'V'),
  ('только', '

Сохраним размеченный POS корпус в файл, для дальнейшего использования.

In [24]:
import json

# открываем выходной файл для записи
with open('data_pos.txt', 'w') as filehandle:  
    json.dump(list_sent_pos, filehandle, indent=4)

Разделим на тест и трэйн

In [25]:
train_data = list_sent_pos[:int(len(list_sent_pos) * 0.9)]
test_data = list_sent_pos[int(len(list_sent_pos) * 0.9):]
test_sent = list_sent_pos[0]
test_data[0]

[('Краснодарский', 'A=m'),
 ('офис', 'S'),
 ('Альфа-банка', 'S'),
 ('заблокировал', 'V'),
 ('семь', 'NUM=acc'),
 ('тысяч', 'S'),
 ('банковских', 'A=pl'),
 ('карт', 'S'),
 ('из-за', 'PR'),
 ('кражи', 'S'),
 ('PIN-кодов', 'S'),
 ('.', 'NONLEX'),
 ('Об', 'PR'),
 ('этом', 'S-PRO'),
 ('сообщает', 'V'),
 ('РИА', 'S'),
 ('Новости', 'S'),
 ('со', 'PR'),
 ('ссылкой', 'S'),
 ('на', 'PR'),
 ('пресс-службу', 'S'),
 ('кредитной', 'A=f'),
 ('организации', 'S'),
 ('.', 'NONLEX'),
 ('В', 'PR'),
 ('Альфа-банке', 'S'),
 ('пояснили', 'V'),
 (',', 'NONLEX'),
 ('что', 'CONJ'),
 ('PIN-коды', 'S'),
 ('стали', 'V'),
 ('известны', 'A=pl'),
 ('мошенникам', 'S'),
 (',', 'NONLEX'),
 ('и', 'CONJ'),
 ('за', 'PR'),
 ('счет', 'S'),
 ('блокировки', 'S'),
 ('карт', 'S'),
 ('``', 'S'),
 ('была', 'V'),
 ('предотвращена', 'V'),
 ('попытка', 'S'),
 ('массового', 'A=n'),
 ('изъятия', 'S'),
 ('денежных', 'A=pl'),
 ('средств', 'S'),
 ('со', 'PR'),
 ('счетов', 'S'),
 ('клиентов', 'S'),
 ("''", 'NONLEX'),
 ('.', 'NONLEX'),
 ('П

### DefaultTagger

In [84]:
default_tagger = nltk.DefaultTagger('S')
default_tagger.evaluate(test_data)

0.35801745348901615

### UnigramTagger

In [36]:
unigram_tagger = UnigramTagger(train_data)

In [80]:
unigram_tagger.evaluate(test_data)

0.9570454424118258

### BigramTagger

In [81]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
bigram_tagger.evaluate(test_data)

0.9625920395416037

### TrigramTagger

In [85]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)

In [86]:
trigram_tagger.evaluate(test_data)

0.9643195841189929

### Combination of taggers

In [77]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

In [78]:
backoff = DefaultTagger('S') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 

In [79]:
tag.evaluate(test_data) 

0.9686610135436644

### Протестируем обученные теггеры на другом тексте (не от новостного портала) и сравним результаты.

In [73]:
reader_corpus_test = WordListCorpusReader('.', ['russian.txt'])

In [66]:
list_sent_test = reader_corpus_test.words()
list_sent_test[:5]

['александр сергеевич пушкин',
 'евгений онегин',
 'роман в стихах',
 '1',
 'мысля гордый свет забавить']

In [67]:
list_sent_pos_test = []
for el in list_sent_test:
    list_sent_pos_test.append(get_pos(el))
list_sent_pos_test[:5]

[[('александр', 'S'), ('сергеевич', 'S'), ('пушкин', 'S')],
 [('евгений', 'A=m'), ('онегин', 'S')],
 [('роман', 'S'), ('в', 'PR'), ('стихах', 'S')],
 [('1', 'NUM=ciph')],
 [('мысля', 'S'), ('гордый', 'A=m'), ('свет', 'S'), ('забавить', 'V')]]

### DefaultTagger

In [68]:
display(default_tagger.tag(list_sent_pos_test[0]), default_tagger.evaluate(list_sent_pos_test))

[(('александр', 'S'), 'S'), (('сергеевич', 'S'), 'S'), (('пушкин', 'S'), 'S')]

0.292742783343632

### UnigramTagger

In [69]:
display(unigram_tagger.tag(list_sent_pos_test[0]), unigram_tagger.evaluate(list_sent_pos_test))

[(('александр', 'S'), None),
 (('сергеевич', 'S'), None),
 (('пушкин', 'S'), None)]

0.8216050160044748

### BigramTagger

In [70]:
display(bigram_tagger.tag(list_sent_pos_test[0]), bigram_tagger.evaluate(list_sent_pos_test))

[(('александр', 'S'), None),
 (('сергеевич', 'S'), None),
 (('пушкин', 'S'), None)]

0.8270345304231449

### TrigramTagger

In [71]:
display(trigram_tagger.tag(list_sent_pos_test[0]), trigram_tagger.evaluate(list_sent_pos_test))

[(('александр', 'S'), None),
 (('сергеевич', 'S'), None),
 (('пушкин', 'S'), None)]

0.8272453853520252

### Combination of taggers

In [ ]:
backoff = DefaultTagger('S') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 

In [87]:
tag.evaluate(list_sent_pos_test) 

0.891585424067333

#### Ожидаемо на "незнакомом" тексте показатели чуть ниже, чем на тексте от новостного портала. 

---

#### Создание тэггера stop words

In [88]:
from nltk.tag import SequentialBackoffTagger
from nltk.corpus import stopwords

In [89]:
class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.stopwords_set = set([n.lower() for n in stopwords.words()])
        
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.stopwords_set:
             return 'stop_W'
        else:
             return None

In [91]:
nt = NamesTagger()
print(nt.tag(['and'])) 
print(nt.tag(['sentens'])) 
print(nt.tag(['the']))            

[('and', 'stop_W')]
[('sentens', None)]
[('the', 'stop_W')]


---